In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

features = 'cpc'
PATH = '/network/tmp1/bakhtias/Results/log_monkeynet_ucf101_path_2_0/ucf101-64_rnet_dpc-rnn_bs30_lr0.001_seq8_pred3_len5_ds3_train-all/model/epoch100.pth.tar'
args = {'features': features,#'airsim_02',
        'PATH': PATH,
        'slowfast_root': '../../slowfast',
        'ntau': 1,
        'subsample_layers': False}

from pretrained_models import get_feature_model

model,_,metadata = get_feature_model(args)

Using DPC-RNN model
final feature map has size 16x16


In [3]:
from pretrained_models import get_model_layers
layers = get_model_layers(model)

layers2viz = [layers[i] for i in range(len(layers)) if 'relu' in layers[i] and 'branch' not in layers[i]]
for i in range(len(layers2viz)):
    print(layers2viz[i])
    


path1_res_blocks_res0_relu
path1_res_blocks_res1_relu
path1_res_blocks_res2_relu
path1_res_blocks_res3_relu
path1_res_blocks_res4_relu
path1_res_blocks_res5_relu
path1_res_blocks_res6_relu
path1_res_blocks_res7_relu
path1_res_blocks_res8_relu
path1_res_blocks_res9_relu
path2_res_blocks_res0_relu
path2_res_blocks_res1_relu
path2_res_blocks_res2_relu
path2_res_blocks_res3_relu
path2_res_blocks_res4_relu
path2_res_blocks_res5_relu
path2_res_blocks_res6_relu
path2_res_blocks_res7_relu
path2_res_blocks_res8_relu
path2_res_blocks_res9_relu
s1_relu


In [5]:
import lucentpatch
import lucentpatch.transform

from lucentpatch.render import render_vis
from lucentpatch.objectives import neuron, slow, tv_slow, intensity_preservation
from lucent.optvis.objectives import channel
from lucent.optvis import param
import lucent
import lucent.optvis
import lucent.optvis.transform

import numpy as np
import torch

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def get_anim(result, size=4, norm=False):
    fig = plt.figure(figsize=(size, size))
    ax = plt.gca()
    im = plt.imshow([[1]])
    plt.axis('off')

    rg = None
    if norm:
        rg = [result.min(), result.max()]

    def drawframe(n):
        if n >= nt:
            n = nt - 1
        else:
            im.set_data(result.transpose((2, 3, 1, 0))[:, :, :, n])
        if norm:
            im.set_clim(rg)
        return (im)

    # blit=True re-draws only the parts that have changed.
    anim = animation.FuncAnimation(fig, drawframe, frames=nt*2, interval=100, blit=False)
    plt.close()
    return anim

#layer_name = 'res0_relu'
for layer_name in layers2viz:
    print(f'*** {layer_name} ***')
# layer_name = 's1_relu'
    for filt in range(64):
        nt = 7
        device = 'cuda'
        batch_param_f = lambda: param.image(60, batch=nt, fft=False)

        # Most important layers: layer2_1_relu, layer3_0_relu, layer3_1_relu
        obj = neuron(layer_name, filt, (1, 1)) #+ 2 * slow('input') + 2 * tv_slow('input') #+ 5 * intensity_preservation('input', 112 // 4, 112)

        model = model.to(device).eval()
        result = render_vis(model, 
                            obj, 
                            batch_param_f, 
                            show_image=False, 
                            show_inline=False, 
                            thresholds=(128,), 
                            fixed_image_size=True,
                            preprocess=False,
                            transforms=[lucentpatch.transform.normalize(),
                                        lucent.optvis.transform.pad(12, mode="constant", constant_value=0.5),
                                        lucent.optvis.transform.jitter(8),
                                        lucent.optvis.transform.random_scale([1 + (i - 5) / 50.0 for i in range(11)]),
                                        lucent.optvis.transform.random_rotate(list(range(-10, 11)) + 5 * [0]),
                                        lucent.optvis.transform.jitter(4),
                                        #lucentpatch.transform.loop(3)
                                        ],
                            )

        anim = get_anim(result[0], 1.5)
        anim.save(f'/network/tmp1/bakhtias/Results/figs/optimal_stim/{features}_{layer_name}_{filt}.gif', writer='imagemagick', fps=12, bitrate=10000)
        # HTML(anim.to_html5_video())

        #base_mask = result[0].max(axis=0).max(axis=0)
        #mask = (base_mask.mean(axis=0, keepdims=True) > .51) * (base_mask.mean(axis=1, keepdims=True) > .51)

  0%|          | 0/128 [00:00<?, ?it/s]

*** path1_res_blocks_res0_relu ***


  2%|▏         | 3/128 [00:00<00:05, 24.53it/s]

*** path1_res_blocks_res1_relu ***


  2%|▏         | 2/128 [00:00<00:07, 17.55it/s]

...

  0%|          | 0/128 [00:00<?, ?it/s]

*** path2_res_blocks_res9_relu ***


  0%|          | 0/128 [00:00<?, ?it/s]

*** s1_relu ***


100%|██████████| 128/128 [00:36<00:00,  3.48it/s]


In [ ]:
from lucent.optvis.param.spatial import pixel_image, fft_image
from lucent.optvis.param.color import to_valid_rgb

import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def deformed_image(w, h=None, sd=None, batch=None, decorrelate=True,
          fft=True, channels=None):
    if sd is None:
        sd = .01
        
    h = h or w
    batch = batch or 1
    ch = channels or 3
    shape = [1, ch, h, w]
    param_f = fft_image if fft else pixel_image
    params, image_f = param_f(shape, sd=sd)
    
    print(shape)
    #d = (torch.randn(1, h, w, 2) * sd).to(device).requires_grad_(True)
    dxy = (torch.randn(1, h // 8, w // 8, 2) * .0000001).to(device).requires_grad_(True)
    
    xi, yi = torch.meshgrid(torch.linspace(-1, 1, h), torch.linspace(-1, 1, h))
    g = torch.stack([yi, xi], dim=2)
    g = g.to(device=device).unsqueeze(0)
    
    def get_output():
        a = image_f()
        outputs = []
        for i in range(batch):
            # Upsample the displacement grid
            dxy_big = F.grid_sample(dxy.permute(0, 3, 1, 2), g, align_corners=True)
            dxy_big = dxy_big.permute(0, 2, 3, 1)
            the_im = F.grid_sample(a, 
                                   g + dxy_big * (i - batch // 2), 
                                   align_corners=True, 
                                   padding_mode='zeros')
            outputs.append(the_im)
        out = torch.cat(outputs, axis=0)
        return out
    
    return params + [dxy], get_output

batch_param_f = lambda: deformed_image(60, batch=nt, fft=False)

result = render_vis(model, 
                    obj, 
                    batch_param_f, 
                    show_image=False, 
                    show_inline=False, 
                    thresholds=(128,), 
                    fixed_image_size=True,
                    preprocess=False,
                    transforms=[],
                    )

#anim.save(f'figures/{layer_name}_{filt}.gif', writer='imagemagick', fps=12, bitrate=10000)
#HTML(anim.to_html5_video())
#anim.min()
#params, output = batch_param_f()
#plt.subplot(121)
#plt.imshow(params[0].cpu().detach().numpy()[0, :, :, :].transpose((1, 2, 0))*10 + .5)
#plt.subplot(122)
#plt.imshow(output().cpu().detach().numpy()[3, :, :, :].transpose((1, 2, 0))*10 + .5)
#params[0].shape

#anim = get_anim(result[0]*.1+.5)
#HTML(anim.to_html5_video())